<a href="https://bing.com">a</a>

In [4]:
from enum import Enum, auto

class Tok(Enum):
    Id   = auto()
    Nm   = auto()
    Grd  = auto()
    Cls  = auto()
    Sid  = auto()
    Bl   = auto()
    Op   = auto()
    Lpar = auto()
    Rpar = auto()
    Assn = auto()
    Must = auto()
    Just = auto()
    Retn = auto()
    Curr = auto()
    New  = auto()
    Eof  = auto()
    
class Token:
    def __init__(self, word):
        self.kind = None
        self.value = None
        
        match word:
            case 'must':
                self.kind = Tok.Must
            case 'just':
                self.kind = Tok.Just
            case 'return':
                self.kind = Tok.Retn
            case 'true' | 'false' as val:
                self.kind = Tok.Bl
                self.value = bool(val.capitalize())
            case 'current':
                self.kind = Tok.Curr
            case 'new':
                self.kind = Tok.New
            case '+' | '-' | '*' | '/' | '<' | '<=' | '==' | '>=' | '>' \
               | '!' | '&&' | '||' as op:
                self.kind = Tok.Op
                self.value = op
            case '&' | 'in':
                self.kind = Tok.Op
                self.value = '&'
            case '=':
                self.kind = Tok.Assn
            case '(':
                self.kind = Tok.Lpar
            case ')':
                self.kind = Tok.Rpar
            case 'eof':
                self.kind = Tok.Eof
            case num if num.isnumeric():
                num = int(num)
                self.value = num
                if num < 2000:
                    self.kind = Tok.Nm
                elif num < 10000:
                    self.kind = Tok.Grd
                elif num < 1000000:
                    self.kind = Tok.Cls
                elif num < 100000000:
                    self.kind = Tok.Sid
            case ident:
                self.kind = Tok.Id
                self.value = ident
                    
    def __str__(self):
            res = str(self.kind)
            if self.value:
                res += f'({self.value})'
            return res
    __repr__ = __str__

class Lexer:
    def __init__(self, line):
        self.tokens = [
            Token(word) for word in line.split()
        ]
        self.pos = 0
    
    def next(self):
        if self.pos == len(self.tokens):
            return Token('eof')
        tok = self.tokens[self.pos]
        self.pos += 1
        return tok
    
    def peek(self):
        if self.pos == len(self.tokens):
            return Token('eof')
        return self.tokens[self.pos]
    
    def __str__(self):
        return str(self.tokens)

    def __repr__(self):
        return repr(self.tokens)
        
def lex(text):
    return [
        Lexer(line)
        for line in text.split('\n') if line and not line.startswith('#')
    ]

### Pratt Parsing
See <a href="https://matklad.github.io">Matklad's amazing post</a>.

In [5]:
class AST:
    def __init__(self, kind):
        self.kind = kind

class Stmt(AST):
    def __init__(self, kind, expr):
        super().__init__(kind)
        self.expr = expr
    
    def __str__(self):
        return f'{self.kind}: {self.expr}'
    __repr__ = __str__

class Assn(AST):
    def __init__(self, ident, expr):
        super().__init__('Assn')
        self.ident = ident
        self.expr = expr
        
    def __str__(self):
        return f'Assn: {self.ident} = {self.expr}'
    __repr__ = __str__

class UniOp(AST):
    def __init__(self, kind, expr):
        super().__init__(kind)
        self.expr = expr
        
    def __str__(self):
        return f'{self.kind}({self.expr})'

class BinOp(AST):
    def __init__(self, kind, lhs=None, rhs=None):
        super().__init__(kind)
        self.lhs = lhs
        self.rhs = rhs
    
    def __str__(self):
        return f'{self.kind}({self.lhs}, {self.rhs})'
    __repr__ = __str__

class Value(AST):
    def __init__(self, kind, value):
        super().__init__(kind)
        self.value = value
        
    def __str__(self):
        return f'{self.kind}({self.value})'
    __repr__ = __str__
        
class Parser:
    def __init__(self, lexers):
        self.ast = []
        for lexer in lexers:
            self.ast.append(self.parse_line(lexer))
        self.ast.append(Stmt('Retn', Value('Bl', True)))
    
    def parse_line(self, lexer):
        tok = lexer.next()
        match tok.kind:
            case Tok.Must | Tok.Just | Tok.Retn:
                expr = self.parse_expr(lexer)
                return Stmt(tok.kind.name, expr)
            case Tok.Id:
                self.eat(lexer, Tok.Assn)
                expr = self.parse_expr(lexer)
                return Assn(tok.value, expr)
                
    def eat(self, lexer, kind):
        if lexer.next().kind != kind:
            raise Error()
                
    def parse_expr(self, lexer):
        return self._parse_expr(lexer, 0)
        
    def _parse_expr(self, lexer, min_bp):
        tok = lexer.next()
        match tok.kind:
            case Tok.Nm | Tok.Id | Tok.Bl | Tok.Curr | Tok.New | Tok.Grd | Tok.Cls | Tok.Sid:
                lhs = Value(tok.kind.name, tok.value)
            case Tok.Op:
                bp = self._prefix_bp(tok.value)
                expr = self._parse_expr(lexer, bp)
                lhs = UniOp(tok.value, expr)
            case Tok.Lpar:
                lhs = self._parse_expr(lexer, 0)
        
        while True:
            op = lexer.peek()
            if op.kind != Tok.Op: break
            
            (lbp, rbp) = self._infix_bp(op.value)
            if lbp < min_bp: break
            
            lexer.next()
            rhs = self._parse_expr(lexer, rbp)
            
            lhs = BinOp(op.value, lhs, rhs)
        
        return lhs

    def _prefix_bp(self, op):
        match op:
            case '+' | '-':
                return 9
            case '!':
                return 3
                
    def _infix_bp(self, op):
        match op:
            case '&&' | '||':
                return (1, 2)
            case '<' | '<=' | '==' | '>=' | '>':
                return (5, 6)
            case '+' | '-':
                return (7, 8)
            case '*' | '/':
                return (11, 12)
            case '&':
                return (13, 14)

In [6]:
class ASTWalker:
    def __init__(self):
        pass
    
    def visit(self, node):
        if isinstance(node, UniOp):
            return self.visit_uniop(node)
        elif isinstance(node, BinOp):
            return self.visit_binop(node)
        elif isinstance(node, Value):
            return self.visit_value(node)
        elif isinstance(node, Stmt):
            return self.visit_stmt(node)
        elif isinstance(node, Assn):
            return self.visit_assn(node)

class VirtualMachine(ASTWalker):
    def __init__(self):
        super().__init__()
        self.var_table = {}
        self.running = True
        self.result = True
        
    def run(self, asts, current, new):
        self.current = [Sid(stu) for stu in current]
        self.new = [Sid(stu) for stu in new]
        
        for ast in asts:
            self.visit(ast)
            
            if not self.running:
                return self.result
    
    def visit_stmt(self, node):
        res = self.visit(node.expr)
        match node.kind:
            case 'Must':
                if not self.check(res):
                    self.running = False
                    self.result = False
            case 'Just':
                if self.check(res):
                    self.running = False
            case 'Retn':
                self.running = False
                self.result = res
    
    def check(self, val):
        if isinstance(val, int) and val == 0:
            return False
        if isinstance(val, bool):
            return val
        return True
                
    def visit_assn(self, node):
        self.var_table[node.ident] = self.visit(node.expr)
    
    def visit_uniop(self, node):
        match node.kind:
            case '+':
                return self.visit(node.expr)
            case '-':
                return -self.visit(node.expr)
            case '!':
                return not self.check(self.visit(node.expr))
    
    def visit_binop(self, node):
        lhs = self.visit(node.lhs)
        rhs = self.visit(node.rhs)
        match node.kind:
            case '+':
                return lhs + rhs
            case '-':
                return lhs - rhs
            case '*':
                return lhs * rhs
            case '/':
                return lhs / rhs
            case '<':
                return lhs < rhs
            case '<=':
                return lhs <= rhs
            case '==':
                return lhs == rhs
            case '>=':
                return lhs >= rhs
            case '>':
                return lhs > rhs
            case '&&':
                return self.check(lhs) and self.check(rhs)
            case '||':
                return self.check(lhs) or self.check(rhs)
            case '&':
                return rhs.has(lhs)
            
    def visit_value(self, node):
        match node.kind:
            case 'Id':
                return self.var_table[node.value]
            case 'Nm' | 'Bl':
                return node.value
            case 'Curr':
                return self.current
            case 'New':
                return self.new
            case 'Grd':
                return Grd(node.value)
            case 'Cls':
                return Cls(node.value)
            case 'Sid':
                return Sid(node.value)

class Grd:
    def __init__(self, value):
        self.grd = value
        
    def has(self, union):
        return all(map(lambda u: u.grd == self.grd, union))
        
class Cls:
    def __init__(self, value):
        self.grd = value // 100
        self.cls = value
        
    def has(self, union):
        return all(map(lambda u: u.cls == self.cls, union))
    
class Sid:
    def __init__(self, value):
        self.grd = value // 10000
        self.cls = value // 100
        self.sid = value

In [7]:
text = '''
# yeah
a = - 10 + 5 * ( 6 + 7 )
must a > 5 && a < 500
must current in 202203
just new in 202205
just 5 <= 10
return false
'''

ast = Parser(lex(text)).ast

print(VirtualMachine().run(ast,
    [
        20220305,
        20220316
    ],
    [
        20210505,
        20220515
    ]))

ast

True


[Assn: a = +(-(Nm(10)), *(Nm(5), +(Nm(6), Nm(7)))),
 Must: &&(>(Id(a), Nm(5)), <(Id(a), Nm(500))),
 Must: &(Curr(None), Cls(202203)),
 Just: &(New(None), Cls(202205)),
 Just: <=(Nm(5), Nm(10)),
 Retn: Bl(True),
 Retn: Bl(True)]